# SpikeInterface pipeline for Tank Lab

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

import spikeextractors as se
import spiketoolkit as st
import spikesorters as ss
import spikecomparison as sc
import spikewidgets as sw
from nwb_conversion_tools.conversion_tools import save_si_object

from tank_lab_to_nwb import TowersProcessedNWBConverter
from tank_lab_to_nwb.convert_towers_task.towersprocessednwbconverter import quick_write

%matplotlib notebook

## 1a) Load AP recordings, LF recordings and TTL signals

In [ ]:
#base_path = Path("/Users/abuccino/Documents/Data/catalyst/brody/A256_bank1_2020_09_30_g0")
#base_data_path = Path("D:/Neuropixels/Neuropixels/A256_bank1_2020_09_30/A256_bank1_2020_09_30_g0")
base_data_path = Path("20210115_NPX_and_behavior/2021_01_15_E105/towersTask_g0")
ap_bin_path = base_data_path / "towersTask_g0_imec0" / "towersTask_g0_t0.imec0.ap.bin"
lf_bin_path = base_data_path / "towersTask_g0_imec0" / "towersTask_g0_t0.imec0.lf.bin"

### Make spikeinterface folders

In [ ]:
recording_folder = Path("20210115_NPX_and_behavior/2021_01_15_E105")
spikeinterface_folder = recording_folder / 'spikeinterface'
spikeinterface_folder.mkdir(parents=True, exist_ok=True)

### (optional) stub recording for fast testing; set to False for running processing pipeline on entire data

In [ ]:
stub_test = False
nsec_stub = 5

In [ ]:
recording_ap = se.SpikeGLXRecordingExtractor(ap_bin_path)
recording_lf = se.SpikeGLXRecordingExtractor(lf_bin_path)

if stub_test:
    recording_ap = se.SubRecordingExtractor(recording_ap, end_frame=int(nsec_stub*recording_ap.get_sampling_frequency()))
    recording_lf = se.SubRecordingExtractor(recording_lf, end_frame=int(nsec_stub*recording_lf.get_sampling_frequency()))

In [ ]:
print(f"Sampling frequency AP: {recording_ap.get_sampling_frequency()}")
print(f"Sampling frequency LF: {recording_lf.get_sampling_frequency()}")      

## Load TTL signals

### (Option 1): Use TTLs from the ap.bin file

In [ ]:
ttl, states = recording_ap.get_ttl_events()
rising_times = ttl[states==1]
print(f"Number of TTL events in ap file: {len(rising_times)}")

In [ ]:
if len(rising_times) > 0:
    start_time = recording_ap.frame_to_time(rising_times[0])
    start_frame_ap = int(recording_ap.time_to_frame(start_time))
    start_frame_lf = int(recording_lf.time_to_frame(start_time))
    print(f"Start frame AP: {start_frame_ap}")
    print(f"Start frame LF: {start_frame_lf}")    
else:
    print("No TTL events found in ap file.")

### (Option 2): Use TTLs from the nidq.bin file

In [ ]:
nidq_file = base_data_path / "towersTask_g0_t0.nidq.bin"

In [ ]:
recording_nidq = se.SpikeGLXRecordingExtractor(nidq_file)

In [ ]:
ttl_nidq, states_nidq = recording_nidq.get_ttl_events()
rising_times_nidq = ttl_nidq[states_nidq==1]
print(f"Number of TTL events in nidq file: {len(rising_times_nidq)}")

In [ ]:
if len(rising_times_nidq) > 0:
    start_time = recording_nidq.frame_to_time(rising_times_nidq[0])
    start_frame_ap = int(recording_ap.time_to_frame(start_time))
    start_frame_lf = int(recording_lf.time_to_frame(start_time))
    print(f"Start frame AP: {start_frame_ap}")
    print(f"Start frame LF: {start_frame_lf}")    
else:
    print("No TTL events found in nidq file.")

### Synchronize recording

In [ ]:
recording_ap_sync = se.SubRecordingExtractor(recording_ap, start_frame=start_frame_ap)
recording_lf_sync = se.SubRecordingExtractor(recording_lf, start_frame=start_frame_lf)

### Inspect signals

In [ ]:
w_ts_ap = sw.plot_timeseries(recording_ap, channel_ids=recording_ap.get_channel_ids()[::4], trange=[100, 105])

In [ ]:
w_ts_lf = sw.plot_timeseries(recording_lf, channel_ids=recording_lf.get_channel_ids()[::4])

## 2) Pre-processing

In [ ]:
apply_cmr = True

In [ ]:
if apply_cmr:
    recording_processed = st.preprocessing.common_reference(recording_ap_sync)
else:
    recording_processed = recording_ap_sync

In [ ]:
num_frames = recording_processed.get_num_frames()

In [ ]:
rates, amps = st.postprocessing.compute_channel_spiking_activity(recording_processed, n_jobs=16, chunk_mb=4000,
                                                                 start_frame=10*30000, end_frame=20*30000, 
                                                                 detect_threshold=8, recompute_info=True, 
                                                                 verbose=True)

In [ ]:
fig, axs = plt.subplots(ncols=2)
sw.plot_activity_map(recording_processed, activity="rate", colorbar=True, ax=axs[0])
sw.plot_activity_map(recording_processed, activity="amplitude", colorbar=True, ax=axs[1])

## 3) Run spike sorters

In [ ]:
sorter_list = [
    'tridesclous',
    #'spykingcircus',
    'herdingspikes',
    #'kilosort2',
]

In [ ]:
# Inspect sorter-specific parameters and defaults
for sorter in sorter_list:
    print(f"{sorter} params description:")
    pprint(ss.get_params_description(sorter))
    print("Default params:")
    pprint(ss.get_default_params(sorter))    

In [ ]:
# user-specific parameters
sorter_params = dict(
    #kilosort2=dict(car=False, n_jobs_bin=12, chunk_mb=4000),
    #ironclust=dict(filter=True),
    tridesclous=dict(n_jobs_bin=12, chunk_mb=4000),
    spykingcircus=dict(filter=True, num_workers=16),
    herdingspikes=dict(filter=True)
)

In [ ]:
sorting_outputs = ss.run_sorters(
    sorter_list=sorter_list, 
    recording_dict_or_list=dict(rec0=recording_processed),
    working_folder=spikeinterface_folder / "working1",
    mode="keep", # change to "keep" to avoid repeating the spike sorting
    sorter_params=sorter_params,
    verbose=True,
    run_sorter_kwargs=dict(raise_error=False)
)

In [ ]:
for result_name, sorting in sorting_outputs.items():
    rec_name, sorter = result_name
    print(f"{sorter} found {len(sorting.get_unit_ids())} units")
    
    # tridesclous sometimes has empty clusters
    active_units = []
    for u in sorting.get_unit_ids():
        if len(sorting.get_unit_spike_train(u)) > 0:
            active_units.append(u)
    
    if len(active_units) < len(sorting.get_unit_ids()):
        sorting_outputs[result_name] = se.SubSortingExtractor(sorting, unit_ids=active_units)
        print(f"{sorter} found {len(active_units)} units after removing empty")

In [ ]:
sorting_tdc = sorting_outputs[('rec0', 'tridesclous')]
sw.plot_unit_templates(recording_processed, sorting_tdc, unit_ids=[11, 21],
                       radius=None)

### (Optional) save individual sorting outputs for later use

In [ ]:
save_si_object("ironclust", sorting_outputs[('rec0', 'ironclust')], spikeinterface_folder,
               cache_raw=False, include_properties=True, include_features=False)

## 4) Post-processing: extract waveforms, templates, quality metrics, extracellular features

### Set postprocessing parameters

In [ ]:
# Post-processing params
postprocessing_params = st.postprocessing.get_postprocessing_params()
pprint(postprocessing_params)

In [ ]:
# (optional) change parameters
postprocessing_params['max_spikes_per_unit'] = 1000  # with None, all waveforms are extracted
postprocessing_params['n_jobs'] = 16  # n jobs
postprocessing_params['chunk_mb'] = 4000  # max RAM usage in Mb
postprocessing_params['verbose'] = True  # max RAM usage in Mb

### Set quality metric list

In [ ]:
# Quality metrics
qc_list = st.validation.get_quality_metrics_list()
print(f"Available quality metrics: {qc_list}")

In [ ]:
# (optional) define subset of qc
qc_list = ['snr', 'isi_violation', 'firing_rate']

### Set extracellular features

In [ ]:
# Extracellular features
ec_list = st.postprocessing.get_template_features_list()
print(f"Available EC features: {ec_list}")

In [ ]:
# (optional) define subset of ec
ec_list = ['peak_to_valley', 'halfwidth']

### Postprocess all sorting outputs

In [ ]:
for result_name, sorting in sorting_outputs.items():
    rec_name, sorter = result_name
    print(f"Postprocessing recording {rec_name} sorted with {sorter}")
    tmp_folder = spikeinterface_folder / 'tmp' / sorter
    tmp_folder.mkdir(parents=True, exist_ok=True)
    
    # set local tmp folder
    sorting.set_tmp_folder(tmp_folder)
    
    # compute waveforms
    waveforms = st.postprocessing.get_unit_waveforms(recording_processed, sorting, **postprocessing_params)
    
    # compute templates
    templates = st.postprocessing.get_unit_templates(recording_processed, sorting, **postprocessing_params)
    
    # comput EC features
    ec = st.postprocessing.compute_unit_template_features(recording_processed, sorting,
                                                          feature_names=ec_list, as_dataframe=True)
    # compute QCs
    qc = st.validation.compute_quality_metrics(sorting, recording=recording_processed, 
                                               metric_names=qc_list, as_dataframe=True)
    
    # export to phy
    if sorter == "kilosort2":
        phy_folder = spikeinterface_folder / 'phy' / sorter
        phy_folder.mkdir(parents=True, exist_ok=True)
        st.postprocessing.export_to_phy(recording_processed, sorting, phy_folder)

### Run phy and load curated data

In [ ]:
!phy template-gui spikeinterface/phy/kilosort2/params.py

In [ ]:
sorting_manual_curated = se.PhySortingExtractor(phy_folder, exclude_cluster_groups=['noise'])

In [ ]:
print(f"Kilosort2 found {len(sorting_manual_curated.get_unit_ids())} units after manual curation")

## 5) Ensemble spike sorting

In [ ]:
if len(sorting_outputs) > 1:
    # retrieve sortings and sorter names
    sorting_list = []
    sorter_names_comp = []
    for result_name, sorting in sorting_outputs.items():
        rec_name, sorter = result_name
        sorting_list.append(sorting)
        sorter_names_comp.append(sorter)
        
    # run multisorting comparison
    mcmp = sc.compare_multiple_sorters(sorting_list=sorting_list, name_list=sorter_names_comp)
    
    # plot agreement results
    w_agr = sw.plot_multicomp_agreement(mcmp)
    
    # extract ensamble sorting
    sorting_ensemble = mcmp.get_agreement_sorting(minimum_agreement_count=2)
    
    print(f"Ensamble sorting among {sorter_list} found: {len(sorting_ensemble.get_unit_ids())} units")

In [ ]:
sw.plot_rasters(sorting_ensemble)

### (Optional) save ensemble output for later use

In [ ]:
save_si_object("sorting_ensemble", sorting_ensemble, spikeinterface_folder,
               cache_raw=False, include_properties=True, include_features=False)

# 6) Automatic curation

In [ ]:
# define curators and thresholds
isi_violation_threshold = 0.5
snr_threshold = 5
firing_rate_threshold = 0.1

In [ ]:
sorting_auto_curated = []
sorter_names_curation = []
for result_name, sorting in sorting_outputs.items():
    rec_name, sorter = result_name
    sorter_names_curation.append(sorter)
    
    # firing rate threshold
    sorting_curated = st.curation.threshold_firing_rates(sorting, duration_in_frames=num_frames,
                                                         threshold=firing_rate_threshold, 
                                                         threshold_sign='less')
    
    # isi violation threshold
    sorting_curated = st.curation.threshold_isi_violations(sorting_curated, duration_in_frames=num_frames,
                                                           threshold=isi_violation_threshold, 
                                                           threshold_sign='greater')
    
    # isi violation threshold
    sorting_curated = st.curation.threshold_snrs(sorting_curated, recording=recording_processed,
                                                 threshold=snr_threshold, 
                                                 threshold_sign='less')
    sorting_auto_curated.append(sorting_curated)
    print(f"{sorter} found {len(sorting_curated.get_unit_ids())} units after auto curation")

# 7) Save to NWB; writes only the spikes

In [ ]:
# The name of the NWBFile from the Vermin output
nwbfile_path = base_data_path / "TowersTask.nwb"

# Choose the sorting extractor from the notebook environment you would like to write to NWB
chosen_sorting_extractor = sorting_outputs[('rec0', 'ironclust')]

se.NwbSortingExtractor.write_sorting(
    sorting=chosen_sorting_extractor,
    save_path=nwbfile_path,
    overwrite=False  # this appends the file
)